In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as de
import os
import sys

import fbprophet

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

import seaborn as sns
sns.set(context='paper', style='darkgrid', rc={'figure.facecolor':'white'}, font_scale=1.2)

ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='paper', style='darkgrid', rc={'figure.facecolor':'white'}, font_scale=1.2)

In [4]:
win_path = 'C:/Users/noname/Desktop/wuzhatn'
mac_path = '/Users/noname/documents/github/wuzhatn'
wrk_path = 'C:/Users/user/Documents/GitHub/wuzhatn'

if os.path.exists(win_path):
    main = win_path
elif os.path.exists(mac_path):
    main = mac_path
else:
    main = wrk_path

raw = main + '/raw'
bank_folder = main[:main.rfind('/')] + '/bank'
sys.path.append(bank_folder)
import mih

years = range(2011,2019,1)
y2u = range(2015,2018,1)
st = '2015-1-1'
ed = '2017-3-31'
# ed = '2017-12-31'

In [5]:
print(wrk_path)
wrk_path.rfind('/')
print(wrk_path[:30])

C:/Users/user/Documents/GitHub/wuzhatn
C:/Users/user/Documents/GitHub


In [6]:
all_ds = {}
hum_ds = {}
for y in y2u:
    all_ds[y] = pd.read_excel(f'{raw}/일별평균대기오염도_{str(y)}.xlsx', usecols=[0,1,6,7])
    all_ds[y].columns = ['dt','loca','pm10','pm25']
    all_ds[y]['loca'] = all_ds[y]['loca'].str[:-1]
    all_ds[y].dropna(axis=1, how='all', inplace=True)
    
    hum_ds[y] = pd.read_excel(f'{raw}/일별기상관측_{str(y)}.xlsx', usecols=[0,1,5])
    hum_ds[y].columns = ['dt','loca','ave_hum']
    all_ds[y].dropna(axis=1, how='all', inplace=True)

# let's just work with gangnam

In [7]:
ds = pd.DataFrame()
for y in y2u:
    t = all_ds[y][all_ds[y]['loca']=='종로'].merge(
        hum_ds[y][hum_ds[y]['loca']=='종로'], on=['dt','loca'], how='outer')
    ds = ds.append(t)
    
ds['dt'] = pd.to_datetime(ds['dt'], format='%Y%m%d')
ds.set_index('dt', inplace=True)
fm2 = pd.date_range(st,ed)
ds = ds.reindex(fm2, fill_value=np.nan)

ds.loc[ds['loca'].isna()==True,'loca'] = '종로'
# for v in ds.columns[ds.isnull().any()]:
#     ds[v] = ds[v].interpolate(method='linear')

ds.head()

,loca,pm10,pm25,ave_hum
2015-01-01,종로,80.0,9.0,41.3
2015-01-02,종로,31.0,13.0,47.0
2015-01-03,종로,42.0,21.0,55.9
2015-01-04,종로,49.0,25.0,68.7
2015-01-05,종로,87.0,41.0,70.7


In [22]:
ds = pd.DataFrame()
for y in y2u:
    t = all_ds[y][all_ds[y]['loca']=='강남'].merge(
        hum_ds[y][hum_ds[y]['loca']=='강남'], on=['dt','loca'], how='outer')
    ds = ds.append(t)
    
ds['dt'] = pd.to_datetime(ds['dt'], format='%Y%m%d')
ds.set_index('dt', inplace=True)
fm2 = pd.date_range(st,ed)
ds = ds.reindex(fm2, fill_value=np.nan)

ds.loc[ds['loca'].isna()==True,'loca'] = '강남'
ds['dtt'] = pd.to_datetime(ds.index)
ds['dtt'] = pd.to_datetime(ds['dtt']).dt.strftime('%Y-%m-%d %H:%M:%S')
ds.set_index('dtt', inplace=True)
ds.tail()

,loca,pm10,pm25,ave_hum
dtt,,,,
2017-03-27 00:00:00,강남,71.0,57.0,67.000000
2017-03-28 00:00:00,강남,87.0,61.0,62.500000
2017-03-29 00:00:00,강남,59.0,41.0,53.200001
2017-03-30 00:00:00,강남,72.0,47.0,50.500000
2017-03-31 00:00:00,강남,53.0,34.0,70.000000


In [25]:
ds['loca'] = np.where((ds['loca']=='강남')|(ds['loca']=='gn'), 'gn', '')
assert ds[ds['loca']!='gn'].shape[0] == 0
display(ds.head())
ds[['loca','pm10']].to_csv(raw + '/testset.csv', index=True, encoding='cp949')
ds = ds[ds.index <= '2017-01-01']
ds[['loca','pm10']].to_csv(raw + '/testset_upto_2016.csv', index=True, encoding='cp949')

,loca,pm10,pm25,ave_hum
dtt,,,,
2015-01-01 00:00:00,gn,66.0,17.0,40.2
2015-01-02 00:00:00,gn,30.0,14.0,46.5
2015-01-03 00:00:00,gn,39.0,22.0,56.0
2015-01-04 00:00:00,gn,40.0,23.0,68.1
2015-01-05 00:00:00,gn,74.0,41.0,71.5


In [ ]:
plt.ylim([0,125])
plt.plot(ds['pm10'])
plt.plot(ds['ave_hum'])

In [ ]:
plt.plot(ds['pm10'])

In [ ]:
plt.subplots(figsize=(25,9))
# plt.subplot(1, 3, 1)
# plt.plot(ds['pm10'])
plt.subplot(1, 3, 1)
plt.plot(ds.loc['2015-1-1':'2015-12-31',['pm10']])
plt.subplot(1, 3, 2)
plt.plot(ds.loc['2016-1-1':'2016-12-31',['pm10']])
plt.subplot(1, 3, 3)
plt.plot(ds.loc['2017-1-1':'2017-12-31',['pm10']])

# straignt up copy from here

In [ ]:
dc = ds.copy()
print(dc.isna().sum())
dc.loc[:,'pm10'] = mih.median_filter(dc, varname='pm10', std=4)
print(dc.isna().sum())
plt.plot(dc['pm10'])

In [ ]:
subs = dc

seas_cycl = subs.loc[:,'pm10'].rolling(window=30, center=True).\
            median().groupby(subs.index.dayofyear).median()
q25 = subs.loc[:,'pm10'].rolling(window=30, center=True).\
            median().groupby(subs.index.dayofyear).quantile(0.25)
q75 = subs.loc[:,'pm10'].rolling(window=30, center=True).\
            median().groupby(subs.index.dayofyear).quantile(0.75)

In [ ]:
from calendar import day_abbr, month_abbr, mdays
ndays_m = mdays.copy()
ndays_m[2] = 29
ndays_m = np.cumsum(ndays_m)
month_abbr = month_abbr[1:]
print(month_abbr)

In [ ]:
f, ax = plt.subplots(figsize=(8,6)) 

seas_cycl.plot(ax=ax, lw=2, color='k', legend=False)

ax.fill_between(seas_cycl.index, q25.values.ravel(), q75.values.ravel(), color='0.8')

ax.set_xticks(ndays_m)
ax.set_xticklabels(month_abbr)

ax.grid(ls=':')

ax.set_xlabel('', fontsize=15)

ax.set_ylabel('cyclists number', fontsize=15);

[l.set_fontsize(13) for l in ax.xaxis.get_ticklabels()]
[l.set_fontsize(13) for l in ax.yaxis.get_ticklabels()]

ax.set_title('Running Average of Daily PM10 Level from 2014 to 2018', fontsize=15)

In [ ]:
dc.head()

In [ ]:
data = dc[['pm10']]
f, ax = plt.subplots(figsize=(14,8))

# plt.plot(data)
# data.plot(ax=ax, color='0.2')
plt.plot(data, color='0.2')
data.rolling(window=30, center=True).mean().plot(ax=ax, ls='-', lw=3, color='0.6')

ax.grid(ls=':')
ax.legend(['daily values','30 days running average'], frameon=False, fontsize=14)

[l.set_fontsize(13) for l in ax.xaxis.get_ticklabels()]
[l.set_fontsize(13) for l in ax.yaxis.get_ticklabels()]

ax.set_xlabel('Date', fontsize=15)
ax.set_ylabel('PM10 Level', fontsize=15);
# ax.axvline('2018-01-01', color='0.8', lw=8, zorder=-1)
data.rename(columns={'pm10':'y', 'index':'ds'}, inplace=True)

In [ ]:
data_train, data_test = mih.prepare_data(data, 2017)
data_train.rename(columns={'index':'ds'}, inplace=True)
data_test.rename(columns={'index':'ds'}, inplace=True)
display(data_train.tail())
display(data_test.head())
display(data_test.tail())

Prophet = fbprophet.Prophet
m = Prophet(mcmc_samples=300, changepoint_prior_scale=0.01, seasonality_mode='multiplicative', \
            yearly_seasonality=12, \
            weekly_seasonality=False, \
            daily_seasonality=False)
# m.add_seasonality(name='monthly', period=30.5, fourier_order=8)

In [ ]:
m.fit(data_train)

In [ ]:
future = m.make_future_dataframe(periods=len(data_test), freq='1D')
future.tail()

In [ ]:
forecast = m.predict(future)

In [ ]:
f = m.plot_components(forecast)

In [ ]:
verif = mih.make_verif(forecast, data_train, data_test)

In [ ]:
f = mih.plot_verif(verif, year=2017)

In [ ]:
print(verif.shape)
print(verif.isna().sum())
verif.head()
verif[verif['y'].isna()==True].head()

In [ ]:
# display(data_train[data_train['ds']=='2014-4-16'])
# display(ds.loc['2014-4-16'])
# display(dc.loc['2014-4-16'])

In [ ]:
print(verif.loc[(verif['ds']>='2016-1-1') & (verif['ds']<='2016-12-31') & \
                (verif['y'].isna()==False),['y']].min())
print(verif.loc[(verif['ds']>='2016-1-1') & (verif['ds']<='2016-12-31') & \
                (verif['y'].isna()==False),['y']].max())

In [ ]:
def plot_joint_plot(verif, x='yhat', y='y', title=None, fpath = '../figures/paper', fname = None):
    """

    Parameters
    ----------
    verif : pandas.DataFrame

    x : string
        The variable on the x-axis
        Defaults to `yhat`, i.e. the forecast or estimated values.

    y : string
        The variable on the y-axis
        Defaults to `y`, i.e. the observed values

    title : string
        The title of the figure, default `None`.

    fpath : string
        The path to save the figures, default to `../figures/paper`

    fname : string
        The filename for the figure to be saved
        ommits the extension, the figure is saved in png, jpeg and pdf

    Returns
    -------
    f : matplotlib Figure object
    """

    g = sns.jointplot(x='yhat', y='y', data = verif, kind="reg", color="0.4")

    g.fig.set_figwidth(8)
    g.fig.set_figheight(8)

    ax = g.fig.axes[1]

    if title is not None:
        ax.set_title(title, fontsize=16)

    ax = g.fig.axes[0]

    ax.set_xlim([-5, None])
#     ax.set_ylim([-5, 3000])
    ax.set_ylim([-5, 200])

#     ax.text(100, 2500, "R = {:+4.2f}\nMAE = {:4.1f}".format(verif.loc[:,['y','yhat']].corr().iloc[0,1], MAE(verif.loc[:,'y'].values, verif.loc[:,'yhat'].values)), fontsize=16)

    ax.set_xlabel("model's estimates", fontsize=15)

    ax.set_ylabel("observations", fontsize=15)

    ax.grid(ls=':')

    [l.set_fontsize(13) for l in ax.xaxis.get_ticklabels()]
    [l.set_fontsize(13) for l in ax.yaxis.get_ticklabels()];

    ax.grid(ls=':')

In [ ]:
# mih.plot_joint_plot(verif.loc[:'2016',:], title='train set', fname='train_set_joint_plot_no_climate')
# mih.plot_joint_plot(verif.loc[(verif['ds']>='2016-1-1') & (verif['ds']<='2016-12-31') & (verif['y'].isna()==False),:], title='train set', fname='train_set_joint_plot_no_climate')
plot_joint_plot(verif.loc[(verif['ds']>='2015-1-1') & (verif['ds']<='2016-12-31') & (verif['y'].isna()==False),:], title='train set', fname='train_set_joint_plot_no_climate')

In [ ]:
plot_joint_plot(verif.loc[(verif['ds']>='2017-1-1') & (verif['y'].isna()==False),:], title='train set', fname='train_set_joint_plot_no_climate')

In [ ]:
verif.loc[:,['y','yhat']].corr()

In [ ]:
verif.tail()

In [ ]:
def make_plot_block(verif, start_date, end_date, ax=None): 
    
    df = verif.loc[start_date:end_date,:]
    df = df[df['y'].isnull()==False]
    df.loc[:,'yhat'].plot(lw=2, ax=ax, color='r', ls='-', label='forecasts')
    ax.fill_between(df.index, df.loc[:,'yhat_lower'], df.loc[:,'yhat_upper'], color='coral', alpha=0.3)
    df.loc[:,'y'].plot(lw=2, ax=ax, color='steelblue', ls='-', label='observations')

    ax.grid(ls=':')
    ax.legend(fontsize=15)
    [l.set_fontsize(13) for l in ax.xaxis.get_ticklabels()]
    [l.set_fontsize(13) for l in ax.yaxis.get_ticklabels()]
    ax.set_ylabel('cyclists number', fontsize=15)
    ax.set_xlabel('', fontsize=15)

    ax.set_title(f'{start_date} to {end_date}', fontsize=18)

In [ ]:
f, axes = plt.subplots(figsize=(17,10), sharey=True)
make_plot_block(verif, '2015-1-1', '2016-3-31', ax=axes)

In [ ]:
f, axes = plt.subplots(figsize=(17,10), sharey=True)
make_plot_block(verif, '2016-1-1', '2017-3-31', ax=axes)

In [ ]:
f, axes = plt.subplots(figsize=(17,10), sharey=True)
make_plot_block(verif, '2016-1-1', '2016-12-31', ax=axes)

In [ ]:
f, axes = plt.subplots(figsize=(17,10), sharey=True)
make_plot_block(verif, '2016-9-1', '2017-3-31', ax=axes)

In [ ]:
f, axes = plt.subplots(figsize=(17,10), sharey=True)
make_plot_block(verif, '2017-1-1', '2017-12-31', ax=axes)

In [ ]:
def make_plot_block(verif, start_date, end_date, ax=None): 
    
    df = verif.loc[start_date:end_date,:]
#     df = df[df['y'].isnull()==False]
#     df.loc[:,'yhat'].plot(lw=2, ax=ax, color='r', ls='-', label='forecasts')
    plt.plot(df['yhat'], lw=2, color='r')
#     df.yhat.plot(lw=2, ax=ax, color='r', ls='-', label='forecasts')
    ax.fill_between(df.index, df.loc[:,'yhat_lower'], df.loc[:,'yhat_upper'], color='coral', alpha=0.3)
    plt.plot(df['y'], lw=2, color='steelblue')
#     df.y.plot(lw=2, ax=ax, color='steelblue', ls='-', label='observations')
    

    ax.grid(ls=':')
    ax.legend(fontsize=15)
    [l.set_fontsize(13) for l in ax.xaxis.get_ticklabels()]
    [l.set_fontsize(13) for l in ax.yaxis.get_ticklabels()]
    ax.set_ylabel('cyclists number', fontsize=15)
    ax.set_xlabel('', fontsize=15)

    ax.set_title(f'{start_date} to {end_date}', fontsize=18)

f, axes = plt.subplots(figsize=(17,10), sharey=True)
make_plot_block(verif, '2018-1-1', '2018-12-31', ax=axes)

In [ ]:
verif.loc['2018',['y','yhat']]